In [ ]:
import pandas as pd
import os
import datetime
from tqdm import tqdm
# from sen_hub import get_all_bands_response
import joblib
from pyproj import CRS

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%load '/content/drive/MyDrive/DataQ/Business development/Products/Jareed/Class/training pipeline/sen_hub.py'

In [ ]:
!pip install sentinelhub
!pip install geopandas

In [ ]:
import geopandas as gpd


In [ ]:
%%time
with open('/content/drive/MyDrive/DataQ/Business development/Products/Jareed/Class/crs.txt', 'r') as file:
    crs = CRS.from_wkt(file.read())


In [ ]:
#load the labels file [year, geometry, Crop_Type, Rainfed, fieldID]

gdf=joblib.load('/content/drive/MyDrive/DataQ/Business development/Products/Jareed/Class/training pipeline/labels.joblib')

In [ ]:
gdf

In [ ]:
years = [2020,2021,2022] #set target years

In [ ]:
# download the bands configured in the senhub config file for each field / Date pairing

gdf['bbox'] = [[r.minx, r.miny, r.maxx, r.maxy] for _,r in gdf.bounds.iterrows()]
#download images for all years available
for year in years:
    capture_dates = [
             ( datetime.datetime(year, 7 , 10).strftime("%Y-%m-%d"), datetime.datetime(year, 7 , 20).strftime("%Y-%m-%d") ),
             ( datetime.datetime(year, 8 , 10).strftime("%Y-%m-%d"), datetime.datetime(year, 8 , 20).strftime("%Y-%m-%d") ),
             #( datetime.datetime(year, 9 , 1).strftime("%Y-%m-%d"), datetime.datetime(year, 9 , 5).strftime("%Y-%m-%d") ),
             ( datetime.datetime(year, 9 , 10).strftime("%Y-%m-%d"), datetime.datetime(year, 9 , 20).strftime("%Y-%m-%d") ),
             #( datetime.datetime(year, 9 , 25).strftime("%Y-%m-%d"), datetime.datetime(year, 9 , 30).strftime("%Y-%m-%d") ),
             #( datetime.datetime(year, 10 , 1).strftime("%Y-%m-%d"), datetime.datetime(year, 10 , 5).strftime("%Y-%m-%d") ),
             ( datetime.datetime(year, 10 , 10).strftime("%Y-%m-%d"), datetime.datetime(year, 10 , 20).strftime("%Y-%m-%d") ),
             #( datetime.datetime(year, 10 , 25).strftime("%Y-%m-%d"), datetime.datetime(year, 10 , 30).strftime("%Y-%m-%d") ),
             #( datetime.datetime(year, 11 , 1).strftime("%Y-%m-%d"), datetime.datetime(year, 11 , 5).strftime("%Y-%m-%d") ),
             ( datetime.datetime(year, 11 , 10).strftime("%Y-%m-%d"), datetime.datetime(year, 11 , 20).strftime("%Y-%m-%d") ),
             #( datetime.datetime(year, 11 , 25).strftime("%Y-%m-%d"), datetime.datetime(year, 11 , 30).strftime("%Y-%m-%d") ),
             ( datetime.datetime(year, 12, 10).strftime("%Y-%m-%d"), datetime.datetime(year, 12, 20).strftime("%Y-%m-%d") )]
    #Download images for all dates (7 (one each month for each field)
    for start_date, end_date in capture_dates:
        output_dir = f'./new_downloads/{year}/{start_date}/'
        if not os.path.exists(output_dir) : os.makedirs(output_dir)
        for i,r in tqdm(gdf.iterrows()):
            if r.Year!=year:
                continue
            output_dir_img = output_dir + f'Field_Id_{r.fieldID}'
            #skip download if the image is downloaded
            if  os.path.exists(output_dir_img):
                print(f'The Image {output_dir_img} is there ')
                continue

            # get configured bands for each field and dat pairing
            img = get_all_bands_response(bbox=r.bbox, start_date=start_date, end_date=end_date,output_dir=output_dir_img, save_data=True)
        #break

In [ ]:
#update the labels gdf by adding image paths to each field.
pathS=[]
for i, x in gdf.iterrows():
    for year in years:
        if x.Year!=year:
                continue
        capture_dates = [
             ( datetime.datetime(year, 7 , 10).strftime("%Y-%m-%d"), datetime.datetime(year, 7 , 20).strftime("%Y-%m-%d") ),
             ( datetime.datetime(year, 8 , 10).strftime("%Y-%m-%d"), datetime.datetime(year, 8 , 20).strftime("%Y-%m-%d") ),
             ( datetime.datetime(year, 9 , 1).strftime("%Y-%m-%d"), datetime.datetime(year, 9 , 5).strftime("%Y-%m-%d") ),
             ( datetime.datetime(year, 9 , 10).strftime("%Y-%m-%d"), datetime.datetime(year, 9 , 20).strftime("%Y-%m-%d") ),
             ( datetime.datetime(year, 9 , 25).strftime("%Y-%m-%d"), datetime.datetime(year, 9 , 30).strftime("%Y-%m-%d") ),
             ( datetime.datetime(year, 10 , 1).strftime("%Y-%m-%d"), datetime.datetime(year, 10 , 5).strftime("%Y-%m-%d") ),
             ( datetime.datetime(year, 10 , 10).strftime("%Y-%m-%d"), datetime.datetime(year, 10 , 20).strftime("%Y-%m-%d") ),
             ( datetime.datetime(year, 10 , 25).strftime("%Y-%m-%d"), datetime.datetime(year, 10 , 30).strftime("%Y-%m-%d") ),
             ( datetime.datetime(year, 11 , 1).strftime("%Y-%m-%d"), datetime.datetime(year, 11 , 5).strftime("%Y-%m-%d") ),
             ( datetime.datetime(year, 11 , 10).strftime("%Y-%m-%d"), datetime.datetime(year, 11 , 20).strftime("%Y-%m-%d") ),
             ( datetime.datetime(year, 11 , 25).strftime("%Y-%m-%d"), datetime.datetime(year, 11 , 30).strftime("%Y-%m-%d") ),
             ( datetime.datetime(year, 12, 10).strftime("%Y-%m-%d"), datetime.datetime(year, 12, 20).strftime("%Y-%m-%d") )]
        paths=[]
        for start_date, end_date in capture_dates:
            path=f'./new_downloads/{year}/{start_date}'
            imagePath=f'{path}/Field_Id_{x.fieldID}'
            imagePath=f'./samer_downloader/{imagePath}/{os.listdir(imagePath+"/")[0]}/response.tiff'
            paths.append(imagePath)
        pathS.append(paths)

gdf['imgPaths']=pathS

In [ ]:
gdf

In [ ]:
joblib.dump(gdf, 'farmsDClabels.joblib')